In [1]:
# Scrape Indeed

In [2]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import splinter
import requests

In [3]:
titles = []
companies = []
salaries = []
locations = []
urls = []

baseurl = 'https://www.indeed.com/jobs?q=data+science&l=California'

for i in range(0, 1010, 10):
    
    url = baseurl + '&start=' + str(i)

    # Retrieve page with the requests module
    response = requests.get(url)
    soup = bs(response.text, 'lxml')
    results = soup.find_all('div',class_='row')

    # Loop through results to retrieve title, company, location, salary, and link
    for result in results:

        #job title
        try:
            job_title = result.a.text.strip()
        except:
            job_title = 'NaN'
        titles.append(job_title)
        
        #company
        try:
            company = result.find(name='span', attrs={'class':'company'}).text.strip()
        except:
            company = 'NaN'
        companies.append(company)

        #location
        try:
            location = result.find(name='div', class_='location').text.strip() 
        except:
            try:
                location = result.find(name='span', class_='location').text.strip()
            except:
                location = 'NaN'

        locations.append(location)

        #salary
        try:
            salary = result.find(name='span', class_='salary no-wrap').text.strip()
        except:
            salary = ' '

        salaries.append(salary)

        try:
            targets = result.find('a', class_='turnstileLink')
            link = 'https://www.indeed.com' + targets.attrs['href']
        except:
            link = ' '
        urls.append(link)


In [4]:
print(len(titles))
print(len(companies))
print(len(locations))
print(len(salaries))
print(len(urls))

1611
1611
1611
1611
1611


In [6]:
indeed_df = pd.DataFrame({
    'Company': companies,
    'Title': titles,
    'Location': locations,
    'Salary': salaries,
    'URL': urls
})

# Delete possible duplicates and clean up
indeed_df.drop_duplicates(subset='URL', inplace=True)
indeed_df.drop_duplicates(subset=['Company','Title','Location'], inplace=True)
indeed_df.dropna()
indeed_df = indeed_df.reset_index(drop=True)

# add column 'Site' and fill in "monster"
indeed_df['Site'] = "Indeed"
# add column 'Keyword'
indeed_df['Keyword'] = "Data Scientist"

# split location into city and state
indeed_df['City'] = indeed_df['Location'].str.split(',', expand=True)[0]
indeed_df['State'] = indeed_df['Location'].str.split(',', expand=True)[1]


In [7]:
indeed_df

,Company,Title,Location,Salary,URL,Site,Keyword,City,State
0,Perspectium,Associate Data Scientist,"San Diego, CA",,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Indeed,Data Scientist,San Diego,CA
1,Harnham US,Data Scientist,"San Diego, CA","$110,000 - $130,000 a year",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Indeed,Data Scientist,San Diego,CA
2,The Control Group,Business Intelligence Data Analyst,"San Diego, CA 92101",,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Indeed,Data Scientist,San Diego,CA 92101
3,Novelty Media,Lead Data Scientist,"La Jolla, CA",,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Indeed,Data Scientist,La Jolla,CA
4,Northrop Grumman,Data Scientist/Principal Data Scientist,"San Diego, CA 92121",,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Indeed,Data Scientist,San Diego,CA 92121
5,Northrop Grumman,Data Scientist 4,"San Diego, CA 92121",,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Indeed,Data Scientist,San Diego,CA 92121
6,University of California - Irvine,Data Scientist,"Irvine, CA 92697","$4,969 - $6,756 a month",https://www.indeed.com/rc/clk?jk=980a2949f46bb...,Indeed,Data Scientist,Irvine,CA 92697
7,Dassault Systèmes,"BIOVIA - Data Science Internship, June - Augus...","San Diego, CA",,https://www.indeed.com/rc/clk?jk=a4ae1626b4e30...,Indeed,Data Scientist,San Diego,CA
8,Xylem,Intern - Data Science - Summer 2019,"San Diego, CA",,https://www.indeed.com/rc/clk?jk=3f21fa41692d4...,Indeed,Data Scientist,San Diego,CA
9,SSF IMPORTED AUTO PARTS LLC,Data Scientist Intern,"San Diego, CA 92121",$15 - $18 an hour,https://www.indeed.com/company/SSF-IMPORTED-AU...,Indeed,Data Scientist,San Diego,CA 92121


In [17]:
# Define a list to store description of each job listing
descs = []
# Counter
i = 0

# Loop through all the urls in the list
for url in indeed_df['URL'].tolist():
    # Create request for page content
    response = requests.get(url)
    #soup = bs(browser.html,'html.parser')
    soup = bs(response.text,'lxml')
    
    # Scrape description only when exists 
    try:
        desc = soup.find('div', class_="jobsearch-JobComponent-description icl-u-xs-mt--md").text
        print(i)
    
    except AttributeError as e:
        desc = " "
    
    # Iteration completed, add 1 to counter
    i += 1
    # Append result to list
    descs.append(desc)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [19]:
len(descs)

814

In [21]:
# Add description and clean up
indeed_df['Description'] =  descs
indeed_df['Description']=indeed_df['Description'].str.strip('\n')
indeed_df

,Company,Title,Location,Salary,URL,Site,Keyword,City,State,Description
0,Perspectium,Associate Data Scientist,"San Diego, CA",,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Indeed,Data Scientist,San Diego,CA,Perspectium is solving enterprise integration ...
1,Harnham US,Data Scientist,"San Diego, CA","$110,000 - $130,000 a year",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Indeed,Data Scientist,San Diego,CA,"$110,000 - $130,000 a yearDATA SCIENTIST\nDATA..."
2,The Control Group,Business Intelligence Data Analyst,"San Diego, CA 92101",,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Indeed,Data Scientist,San Diego,CA 92101,Business Intelligence Data Analyst\n\nThe Cont...
3,Novelty Media,Lead Data Scientist,"La Jolla, CA",,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Indeed,Data Scientist,La Jolla,CA,"Novelty Media is looking for a driven, analyti..."
4,Northrop Grumman,Data Scientist/Principal Data Scientist,"San Diego, CA 92121",,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Indeed,Data Scientist,San Diego,CA 92121,Northrop Grumman is a leading global security ...
5,Northrop Grumman,Data Scientist 4,"San Diego, CA 92121",,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Indeed,Data Scientist,San Diego,CA 92121,Northrop Grumman is a leading global security ...
6,University of California - Irvine,Data Scientist,"Irvine, CA 92697","$4,969 - $6,756 a month",https://www.indeed.com/rc/clk?jk=980a2949f46bb...,Indeed,Data Scientist,Irvine,CA 92697,"$4,969 - $6,756 a monthContractUnder the gener..."
7,Dassault Systèmes,"BIOVIA - Data Science Internship, June - Augus...","San Diego, CA",,https://www.indeed.com/rc/clk?jk=a4ae1626b4e30...,Indeed,Data Scientist,San Diego,CA,InternshipImagine new horizons\n\nAs a Data Sc...
8,Xylem,Intern - Data Science - Summer 2019,"San Diego, CA",,https://www.indeed.com/rc/clk?jk=3f21fa41692d4...,Indeed,Data Scientist,San Diego,CA,"Temporary, InternshipXylem (XYL) is a leading ..."
9,SSF IMPORTED AUTO PARTS LLC,Data Scientist Intern,"San Diego, CA 92121",$15 - $18 an hour,https://www.indeed.com/company/SSF-IMPORTED-AU...,Indeed,Data Scientist,San Diego,CA 92121,$15 - $18 an hourInternshipCompany Overview: S...


In [22]:
#indeed_df.to_csv("Data/indeed_data_science.csv")